# Ticket Tracker - Reduzindo o sofrimento da compra de ingressos

## Índice

## Introdução

## Problema 

## Extração de dados 

In [1]:
# Adicionando dependências

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Beautiful Soup imports
import bs4
from bs4 import BeautifulSoup

import utils
import random
import requests

In [2]:
# Creating a selenium driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_page_load_timeout(60)

In [3]:
# Accessing blacktag
url = 'https://blacktag.com.br/eventos?cidade=S%C3%A3o+Paulo'

# Using random user agents to avoid host suspition
u_agent = random.choice(utils.user_agents)

arguments = [
    "--disable-cookies",
    "--disable-local-storage",
    "--disable-session-storage",
    "--block-third-party-cookies"
    f"user-agent={u_agent}"
]

chrome_options = Options()
for arg in arguments:
    chrome_options.add_argument(arg)

driver = webdriver.Chrome(options=chrome_options)

driver.get(url=url)
driver.implicitly_wait(10)

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

tags = soup.find_all("a", class_ = "w-100")

links = [tag.get("href") for tag in tags]
links

['/eventos/20307/shiva-shankar-warm-up',
 '/eventos/20422/meio-medico-sao-camilo-the-great-29',
 '/eventos/20481/blocckstar-convida',
 '/eventos/19995/mc-ig-na-crazy',
 '/eventos/20236/sabado-noite-open-duble-27-04',
 '/eventos/20655/trifreixoto-festa-da-senha',
 '/eventos/20267/baile-de-gala-unip-saude-4073',
 '/eventos/15301/baile-de-gala-fuzue',
 '/eventos/20586/open-double-premium-vespera-de-feriado-c-persanto',
 '/eventos/20284/arraxta-pra-cima',
 '/eventos/20180/deixa-em-off-x-festa-do-chefe',
 '/eventos/20397/fusion']

## Análise

## Conclusão